# Lecture 01 : On PyTorch

written by SinsuSquid (bgkang) on 00 November 2024

## Intro

저번 시간에는 perceptron, multi-layered perceptron (MLP)의 개념에 대해서만 간단하게 소개했고, 이를 PyTorch Framework를 통해 구현하는 것을 해봤었죠. 하지만 실상 까놓고 보면 우리가 한거라곤 Neural Network (NN) 구성을 위한 Layer 몇개 쌓아놓았던게 끝이였던 것 같아요. 오늘은 본격적으로 PyTorch Framework를 이용해서 model을 생성하고 training/validation & test cycle을 실제로 한번 구현해보는걸 목표로 진행할거에요. 우리가 model을 만든다고 하면 으레 진행하는 cycle이니까 이 과정에 대한 detail한 설명까지는 못해줄 것 같네요.

## Target

그냥 model에 냅다 들이받는 것 보다는 그래도 뭐라도 예측하고자 하는 target이 있으면 좋잖아요? 이번 시간에는 [AqSolDB](https://www.kaggle.com/datasets/sorkun/aqsoldb-a-curated-aqueous-solubility-dataset)에 있는 정보를 사용해서 compound의 solubility를 예측하는 regression model을 학습하고 검증해보도록 하겠습니다. (잊지 마요, 우린 사실 '화학과' 소속이란걸.) 그냥 파일 다운로드 받아도 무방하지만, 기왕 할거 간드러지게 해야 멋있잖아요? kaggle API를 사용해서 AqSolDB를 내 컴퓨터로 다운받아보도록 하겠습니다. (초기 setup은 귀찮으니까 수업할때 얘기할게요) 

In [ ]:
!kaggle datasets list -s AqSolDB # kaggle API를 이용한 dataset searching
!kaggle datasets download -d sorkun/aqsoldb-a-curated-aqueous-solubility-dataset # dataset download
!unzip -u ./aqsoldb-a-curated-aqueous-solubility-dataset.zip # 압축 풀기
!rm ./aqsoldb-a-curated-aqueous-solubility-dataset.zip # 불필요한 zip 파일은 제거
!ls # *.csv 파일이 생성되었나 확인

Dataset이 어떻게 생겨먹었나 한번 살펴볼까요?

In [ ]:
import pandas as pd
df = pd.read_csv('./curated-solubility-dataset.csv')
print(df.columns)
df.head()

더 재미있는 방법으로 dataset을 augment를 시키는것도 가능하지만, 지금 당장은 그럴 의욕이 없네요. 주어진 dataset에서 필요한 부분만 추려서 정리를 해보겠습니다.

In [ ]:
df = df.loc[:,['Solubility', 'MolWt', 'MolLogP', 'MolMR', 'HeavyAtomCount', 'NumHAcceptors',
               'NumHDonors', 'NumHeteroatoms', 'NumRotatableBonds', 'NumValenceElectrons',
               'NumAromaticRings', 'NumSaturatedRings', 'NumAliphaticRings', 'RingCount', 'TPSA',
               'LabuteASA', 'BalabanJ', 'BertzCT']]
df.head()

In [ ]:
import numpy as np
train, validation, test = np.split(df.sample(frac = 1, random_state = 42),
                                   [int(0.8 * len(df)), int(0.9 * len(df))])
print(f"Train : {len(train)} | Validate : {len(validation)} | Test : {len(test)}")

데이터 불러와서 train, validate, test set으로 분리시키는 것 까지 완료했네요. 혹시 왜 이런 일을 하고있는지 이해가 가지 않는다면, 인공지능 기초를 YouTube를 통해 공부해오는게 좋을거에요! 여기까지 가르쳐줄 생각이 없거든요!

## Step 0 : Model Definition

Model을 정의하는 부분부터 시작해볼까요? 저번 시간에 간단하게만 언급했지만 PyTorch의 장점은 "Pythonic"한 방식으로 model을 구성할 수 있다는 점이죠. 강요하진 않겠지만 저는 "Pythonic"하게 내 방식으로 진행할터이니 알아서 잘 따라 오던가 하세요?

In [ ]:
import torch

class LinearReLU(torch.nn.Module):
    def __init__(self, in_size, out_size):
        super().__init__()
        
        self.in_size, self.out_size = in_size, out_size
        
        self.linear = torch.nn.Linear(self.in_size, self.out_size)
        self.relu = torch.nn.ReLU()

        return

    def forward(self, x):
        x = self.linear(x)
        return self.relu(x)

    def reset_parameters(self):
        self.linear.reset_parameters()

class MyModel(torch.nn.Module):
    def __init__(self, in_size, out_size, hidden_size, num_hidden_layer):
        super().__init__()

        self.in_size, self.out_size, self.hidden_size = in_size, out_size, hidden_size
        self.num_hidden_layer = num_hidden_layer


        self.in_layer = LinearReLU(self.in_size, self.hidden_size)
        self.hidden_layer = torch.nn.Sequential(
            *[LinearReLU(self.hidden_size, self.hidden_size) for _ in range(num_hidden_layer)]
        )
        self.out_layer = torch.nn.Linear(self.hidden_size, self.out_size)

    def forward(self, x):
        x = self.in_layer(x)
        for hl in self.hidden_layer:
            x = hl(x)
        return self.out_layer(x)

    def reset_parameters(self):
        self.in_layer.reset_parameters()
        for hl in self.hidden_layer:
            hl.reset_parameters()
        self.out_layer.reset_parameters()

In [ ]:
model = MyModel(in_size = 17, out_size = 1, hidden_size = 50, num_hidden_layer = 3)
model.reset_parameters()
print(model)

## Step 1 : How to use `DataSet` & `DataLoader`

PyTorch 쓰는 이유중에 하나는 강력한 `DataSet`과 `DataLoader`의 기능이죠. 이쯤에서 Python의 `Iterator`라는 개념을 알고 계시면 좋겠네요. DataLoader를 잘 구성하면 epoch마다 model에 batch를 넣어주는 과정을 `Iterator`를 이용해 단순화시킬 수 있습니다. 사용법도 매우 쉬우니까 `DataLoader`의 개념을 잘 이해해 두었다가, 나중에 `torch_geometric`에 대한 설명을 할 때 활용했으면 좋겠어요. (네, 잘 기억해놓으란 뜻입니다.)
<br>\
이제 Custom `Dataset`을 구성하는 부분부터 시작해볼까요? 이 부분은 personal preference가 많이 들어갈 것 같은데, 제 입맛대로 한번 해보도록 할게요.

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df

        self.y = torch.tensor(self.df.iloc[:,0].values) # 'Solubility' column
        self.x = torch.tensor(self.df.iloc[:,1:].values) # anything other than 'Solubility' column

        self.y = self.y.float().reshape(-1, 1) # Type Casting & Formatting
        self.x = self.x.float().reshape(-1, 17)

        return

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        return self.x[idx,:], self.y[idx]

train_dataset = CustomDataset(train)
validation_dataset = CustomDataset(validation)
test_dataset = CustomDataset(test)

In [ ]:
print(len(train_dataset))
print(train_dataset.y)
print(train_dataset[42]) # 이렇게 쓰면 돼요

다음은 `Dataset`으로부터 `DataLoader`를 구성해보도록 할게요. `Dataset`만 멀쩡하게 구성되어있으면 어렵지 않습니다. 오히려 이번에는 `DataLoader`를 어떻게 사용하는지를 조금 더 집중해서 살펴보아요.

In [ ]:
# batch_size에 대한 개념은 알아서 기억해오세요~
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 256)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size = 256)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 256)

In [ ]:
for x, y in test_loader:
    print(x.shape, y.shape) # DataLoader를 쓰면 알아서 batch가 만들어진다는 사실~!

batch를 `Iterator`의 형태로 사용할 수 있다는 건 크나큰 장점이죠. (왜 장점인지 모르겠다고요? 조금만 기다리면 `torch_geometric`에서 알 수 있어요)

## Step 2 : Define training functions

흔히 모델 학습을 할 때 "예측값 계산 -> 가중치 업데이트"를 반복하게 되죠. (네, 알아서 찾아오세요.)
<br>\
PyTorch를 사용하는 사람들은 각 과정을 function으로 정의하여 사용한답니다. (저는 별도의 `Training` Object를 만드는걸 선호하는 편인데, 오늘은 대세에 따라보기로 하죠.)
<br>\
지금부터는 model 학습 과정에서 실제로 어떤 일들이 일어나는지를 정의하는 부분인데, 이 과정을 제가 하나하나 설명할 수는 없습니다. (누가 돈이라도 주면 몰라요 히히) 그러니까 "이런게 왜 필요하지?" 라고 생각이 든다면 이 Lecture를 볼게 아니라 당장 가서 다른 책이나 YouTube를 보는게 맞다고 생각해요. 제가 여기서 해줄 수 있는건 code에서 각 문장이 어떤 과정을 의미하는지만을 알려줄 수 있을 뿐이고, 각각의 procedure들은 꼭 공부를 해오든 찾아오든 할 수 있길 바랍니다.

먼저 `optimizer`나 `criterion` (loss function을 이렇게 부르시더라고요) 등에 대한 정의가 필요한데, 이런 개념들도 스스로 공부를 해오셔야 할 듯 합니다. 이번 Lecture는 PyTorch의 사용법을 익히는게 목표니까, 소위 '국룰'이라 불리는 설정으로 초기화를 해보겠습니다.

In [ ]:
# Adam Optimizer가 뭔지는 여기서 못 알려줘요!
optimizer = torch.optim.Adam(model.parameters(), lr = 1E-3)
# Regression 문제니까 MSE를 사용해요 - 이해 안되면 공부해와잇!
criterion = torch.nn.MSELoss()

In [ ]:
# epoch별 training 단계

def train(model, loader):
    model.train() # model을 train 모드로 설정 - 가중치 업데이트 활성화

    tot_loss = 0.0
    
    for x, y in loader:
        optimizer.zero_grad() # Optimizer를 (일종의) 초기화
        
        out = model(x) # 출력값 계산
        loss = criterion(out, y) # loss 계산
        loss.backward() # 가중치 계산 - 이게 code 한줄로 구현이 가능하다니!!
        
        # (여기서 감동하면 됩니다.)
        
        optimizer.step() # 가중치 update

        tot_loss += loss.item() # Trainng Loss 계산을 위하여

    return tot_loss / len(loader) # averaged 된 loss값을 return

In [ ]:
# epoch별 validation 단계

def validation(model, loader):
    model.eval() # model을 eval 모드로 설정 - 가중치 업데이트 비활성화

    tot_loss = 0.0

    with torch.no_grad(): # gradient 계산 안하겠다 - 그만큼 속도를 아끼겠죠?
        for x, y in loader:
            out = model(x) # 출력값 계산
            loss = criterion(out, y) # loss 계산
    
            tot_loss += loss.item() # Validation Loss 계산을 위하여
    
    return tot_loss / len(loader) # averaged 된 loss값을 return

`validation`에서 확인해야 할 부분은, `loss.backward()`랑 `optimizer`를 호출한적이 없다는 점이에요. 사실 `validation`함수가 수행하는 목적을 생각하면 당연한 일이겠지요? (네, 알아서 이해 해오세요.)
<br>\
마지막으로, epoch하나당 수행해야 할 일을 `run`이란 이름의 함수로 정의해보죠.

In [ ]:
trn_loss = []; val_loss = [] # Loss Curve도 그려봐야하니까~

def run(epochs): # epoch 수를 이 부분에서 받아오도록 하죠
    for e in range(epochs):
        trn_loss.append(train(model, train_loader))
        val_loss.append(validation(model, validation_loader))

        print(f"Epoch : {e:05d} | Trn. Loss : {trn_loss[-1]:.3f} | Val. Loss : {val_loss[-1]:.3f}")

    print("Training Complete! >:D")

## Step 3 : Roll the dice

여기까지 했으면 이제 뭐 하겠어요, 잘 돌아가나 함 봐야지.

In [ ]:
run(100) # 간단하게 100번만?

## Step 4 : Model Validation

다행히 training process까지 무사하게 진행했으면, 이제 실제로 학습된 model의 performance가 어떻게 되는지 체크해봐야겠죠. `test` 혹은 `eval`이란 별도의 함수를 작성하시는 분들도 있지만, (네, 저요.) 하는 짓은 아래 code에서 크게 다르지 않으니까 입맛대로 수정할 수 있을거에요.

In [ ]:
with torch.no_grad():
    model.eval()

    test_loss = 0.0
    pred = []; true = []

    for x, y in test_loader:
        # torch.tensor보다 numpy.ndarray가 다루기 편하니까 변환해서 저장하도록 할게요.
        out = model(x)
        test_loss += criterion(out, y)
        pred.append(out.numpy())
        true.append(y.numpy())

test_loss = test_loss.item() / len(test_loader)
pred = np.concatenate(pred).flatten()
true = np.concatenate(true).flatten()

In [ ]:
test_loss # 이것만으로는 학습이 잘 된건지 모르겠잖아!

model 성능을 더 확인해보기에 앞서, 최소한 학습은 되고 있는가 loss curve를 그려보도록 하죠. (혹시 loss curve가 뭔지 잘 모르겠으면... 알죠?)
<br><br>
이건 여담인데, 저도 그림 그리는 센스가 별로 좋은 편이 아니기 때문에 더 예쁜 그림을 그리고싶으면 다른 자료를 잘 공부하는게 좋아요.

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1, figsize = (8, 8), dpi = 200)

ax.set_xlabel("Epoch", fontsize = 15)
ax.set_ylabel("Loss", fontsize = 15)
ax.set_title("Loss Curve", fontsize = 20)

ax.plot(range(100), trn_loss, linewidth = 3, label = 'Training Loss')
ax.plot(range(100), val_loss, linewidth = 3, label = 'Validation Loss')
ax.hlines(test_loss, 0, 100, linestyle = '--', color = 'k', label = 'Test Loss')

ax.legend(fontsize = 15)
plt.show()

썩 마음에 드는 결과는 아니지만, 우리의 첫 성과잖아요? (보잘 것 없고... 귀여워...)
<br><br>
다음엔 parity plot을 그려보도록 하죠. (이젠 무슨 말 할지 알죠?)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (8, 8), dpi = 200)

ax.set_xlabel("True Value", fontsize = 15)
ax.set_ylabel("Predicted Value", fontsize = 15)
ax.set_title("Solubility", fontsize = 20)

ax.plot(true, true, linestyle = '--', color = 'k', linewidth = 3)
ax.plot(true, pred, linestyle = 'None', marker = 'o', markersize = 7, alpha = 0.5)

plt.show()

이렇게 그림만 그려놓으면 정량적인 비교가 어렵잖아요? Error 지표 몇개정도 계산해볼게요.

In [ ]:
# Mean Squared Error
mae = np.abs(true - pred).mean()

# Root Mean Squared Error
rmse = np.sqrt(np.power(true - pred, 2).mean())

# Mean Absolute Percent Error
mape = np.abs((true - pred) / true * 100).mean()

# R2 - 귀찮으니까 ㅎㅎ
from sklearn.metrics import r2_score
r2 = r2_score(true, pred)

print(f"MAE : {mae:.3f} | RMSE : {rmse:.3f} | MAPE : {mape:.3f} | R2 : {r2:.3f}")

여기까지 왔으면 일반적으로 수행하는 training / validation / test "Cycle"을 한번 수행해본게 되겠네요. (네, "Cycle"이요. 다시 말해, 이걸 밥먹듯이 해야한다는 의미죠.)

## Step 4.5 : Model Save

물론 지금까지 학습된 model은 작고 귀여운 감자에 불과하지만, 그래도 기념이니까 한번 저장을 해보도록 할게요. PyTorch에서는 model의 `state_dict`를 저장하는것을 '권장'하는데, 이는 model의 구조까지는 저장하지 않고 각 layer의 learnable(trainable) parameter만을 저장한다고 생각하면 될 것 같아요. 다시 말해, 저장한 model을 다시 불러 쓰기 위해서는 저장한 model과 동일한 형태 - size 등 - 를 가져야 한다는 뜻입니다.

In [ ]:
PATH = "./my_little_cute_first_potato.model"
torch.save(model.state_dict(), PATH) # 나의 작고 귀여운 감자가 저장되었어요!

In [ ]:
# model과 new_model은 완벽하게 동일한 구조여야 해요!
new_model = MyModel(in_size = 17, out_size = 1, hidden_size = 50, num_hidden_layer = 3)
new_model.load_state_dict(torch.load(PATH, weights_only = True))
new_model

In [ ]:
# Let's Check!
print("Old Potato : ", model.in_layer.linear.weight[0])
print("New Potato : ", new_model.in_layer.linear.weight[0]) # Yay! Another cute little potato!

## Step 5 : Recap

원래 여기까지 계산을 다 완료했으면 다음 단계는 실제 model의 성능을 비교하는 일이 될거에요. 아쉽게도 [AqSolDB의 kaggle 페이지](https://www.kaggle.com/datasets/sorkun/aqsoldb-a-curated-aqueous-solubility-dataset/data)를 확인해보니 성능을 비교할만한 다른 model들이 아직 업로드되지가 않았네요. 하지만 실제 model 개발은 이 단계부터 시작해야해요. 내가 만든 model의 성능은 기존의 model에 비해 어떤 정도인지, 정확도를 높이기 위해서는 어떤 방식으로 개선을 해야하는지 - 새로운 개념? 효율적인 hyperparamter optimization? 등을 고려하며 내 model을 좀더 멋지게 만들어나갈 전략에 대해 생각해보아야겠죠. 뭐, 오늘은 작교 귀여운 model을 처음 만들어본거니 이쯤에서 수고했어요 짝짝짝 한 후 마무리하도록 합시다.

## Outtro

저번 시간부터 밥먹듯이 하는 말이 있죠? 모르는 부분 있으면 알아서 공부 해오라고. 나도 다 아는 내용이고 알려줄 수도 있어요. 내가 여기까지 해 줄 필요가 없어서 (그리고 귀찮아서...) 안하는거에요. 시작하기 전부터 경고 많이 했었지만 들꽃반 만만하게 가르칠 생각 없어요 ㅎㅎ. 대신 언제든지 중도하차는 기쁜 마음으로 허락할게요, 진짜! 마음의 상처 하나도 없이!
<br>
이번 시간 내용이 정리되지 않는다면 다음시간 (아마 `torch_geometric`이겠죠?) 수업을 듣는 이유가 하나도 없을거라고 미리 겁을 좀 더 줄게요. (당근은 안주고 채찍만 계속 주니, 나쁜 기수로군요.)